### Planing with Code

[Source](https://learn.deeplearning.ai/courses/agentic-ai) 

Planning with code execution means letting the LLM write code that becomes the plan itself. Compared to plain-text or JSON-based plans, this approach is more expressive and flexible: the code not only documents the steps but can also execute them directly.

In [40]:
from __future__ import annotations
import json
from dotenv import load_dotenv
from openai import OpenAI
import re, io, sys, traceback, json
from typing import Any, Dict, Optional
from tinydb import Query, where
from typing import Any
import random
from datetime import datetime
from tinydb import TinyDB
from tinydb.storages import MemoryStorage


load_dotenv()
client = OpenAI()

### Database Setup 

In [41]:
# Initialize TinyDB
db = TinyDB("store_db.json")
inventory_table = db.table("inventory")
transactions_table = db.table("transactions")


def create_inventory():
    """
    Create and store the initial sunglasses inventory in TinyDB.
    Each item has name, item_id, description, quantity_in_stock, and price.
    """
    random.seed(42)

    sunglasses_data = [
        {
            "item_id": "SG001",
            "name": "Aviator",
            "description": "Originally designed for pilots, these teardrop-shaped lenses with thin metal frames offer timeless appeal. The large lenses provide excellent coverage while the lightweight construction ensures comfort during long wear.",
            "quantity_in_stock": random.randint(3, 25),
            "price": 80
        },
        {
            "item_id": "SG002",
            "name": "Wayfarer",
            "description": "Featuring thick, angular frames that make a statement, these sunglasses combine retro charm with modern edge. The rectangular lenses and sturdy acetate construction create a confident look.",
            "quantity_in_stock": random.randint(3, 25),
            "price": 95
        },
        {
            "item_id": "SG003",
            "name": "Mystique",
            "description": "Inspired by 1950s glamour, these frames sweep upward at the outer corners to create an elegant, feminine silhouette. The subtle curves and often embellished temples add sophistication to any outfit.",
            "quantity_in_stock": random.randint(3, 25),
            "price": 70
        },
        {
            "item_id": "SG004",
            "name": "Sport",
            "description": "Designed for active lifestyles, these wraparound sunglasses feature a single curved lens that provides maximum coverage and wind protection. The lightweight, flexible frames include rubber grips.",
            "quantity_in_stock": random.randint(3, 25),
            "price": 110
        },
        {
            "item_id": "SG005",
            "name": "Classic",   # renamed from "Round"
            "description": "Classic round profile with minimalist metal frames, offering a timeless and versatile style that fits both casual and formal wear.",
            "quantity_in_stock": random.randint(3, 25),
            "price": 60  # under $100
        },
        {
            "item_id": "SG006",
            "name": "Moon",  # new entry
            "description": "Oversized round style with bold plastic frames, evoking retro aesthetics with a modern twist.",
            "quantity_in_stock": random.randint(3, 25),
            "price": 120  # over $100
        }
    ]

    inventory_table.truncate()
    inventory_table.insert_multiple(sunglasses_data)
    return sunglasses_data


def create_transactions(opening_balance=500.00):
    """
    Create and store the initial transactions in TinyDB.
    Includes the daily opening balance.
    """
    opening_transaction = {
        "transaction_id": "TXN001",
        "customer_name": "OPENING_BALANCE",
        "transaction_summary": "Daily opening register balance",
        "transaction_amount": opening_balance,
        "balance_after_transaction": opening_balance,
        "timestamp": datetime.now().isoformat()
    }

    transactions_table.truncate()
    transactions_table.insert(opening_transaction)
    return opening_transaction


def seed_db(db_path="store_db.json"):
    db = TinyDB(db_path)
    inventory_table = db.table("inventory")
    transactions_table = db.table("transactions")
    create_inventory()       # llena inventory_table
    create_transactions()    # llena transactions_table
    return db, inventory_table, transactions_table


# ==== Schema helpers for TinyDB ====
def _shorten(v: Any, n: int = 60) -> str:
    s = str(v)
    return s if len(s) <= n else s[:n-1] + "…"


def infer_type(value: Any) -> str:
    """Infer a simple type string from a Python value."""
    if isinstance(value, bool):
        return "bool"
    if isinstance(value, int):
        return "int"
    if isinstance(value, float):
        return "float"
    return "string"


def build_schema_for_table(tbl, table_name: str, k: int = 3) -> str:
    rows = tbl.all()
    if not rows:
        return f"TABLE: {table_name} (empty)"
    
    # Infer simple types + take some examples
    schema = {}
    for r in rows:
        for k_, v in r.items():
            if k_ not in schema:
                schema[k_] = {"type": type(v).__name__, "examples": []}
            if len(schema[k_]["examples"]) < k and v not in schema[k_]["examples"]:
                schema[k_]["examples"].append(str(v))

    lines = [f"TABLE: {table_name}", "COLUMNS:"]
    for col, info in schema.items():
        ex = f" | examples: {info['examples']}" if info["examples"] else ""
        lines.append(f"  - {col}: {info['type']}{ex}")
    lines.append(f"ROWS: {len(rows)}")
    lines.append(f"PREVIEW (first 3 rows): {rows}")
    return "\n".join(lines)


def build_schema_block(inventory_tbl, transactions_tbl) -> str:
    inv = build_schema_for_table(inventory_tbl, "inventory_tbl")
    tx = build_schema_for_table(transactions_tbl, "transactions_tbl")
    notes = (
        "NOTES:\n"
        "- inventory_tbl.price is in USD.\n"
        "- inventory_tbl.quantity_in_stock > 0 means available stock.\n"
        "- inventory_tbl.name describes the style (e.g., 'Classic', 'Moon').\n"
        "- transactions_tbl.timestamp is ISO-8601.\n"
    )
    return f"{inv}\n\n{tx}\n\n{notes}"


# ==== Helpers for transactions ====
def get_current_balance(transactions_tbl, default: float = 0.0) -> float:
    txns = transactions_tbl.all()
    return txns[-1].get("balance_after_transaction", default) if txns else default


def next_transaction_id(transactions_tbl, prefix: str = "TXN") -> str:
    return f"{prefix}{len(transactions_tbl)+1:03d}"

In [42]:
db, inventory_tbl, transactions_tbl = seed_db()

In [43]:
inventory_table.all()

[{'item_id': 'SG001',
  'name': 'Aviator',
  'description': 'Originally designed for pilots, these teardrop-shaped lenses with thin metal frames offer timeless appeal. The large lenses provide excellent coverage while the lightweight construction ensures comfort during long wear.',
  'quantity_in_stock': 23,
  'price': 80},
 {'item_id': 'SG002',
  'name': 'Wayfarer',
  'description': 'Featuring thick, angular frames that make a statement, these sunglasses combine retro charm with modern edge. The rectangular lenses and sturdy acetate construction create a confident look.',
  'quantity_in_stock': 6,
  'price': 95},
 {'item_id': 'SG003',
  'name': 'Mystique',
  'description': 'Inspired by 1950s glamour, these frames sweep upward at the outer corners to create an elegant, feminine silhouette. The subtle curves and often embellished temples add sophistication to any outfit.',
  'quantity_in_stock': 3,
  'price': 70},
 {'item_id': 'SG004',
  'name': 'Sport',
  'description': 'Designed for a

#### Database Schema:
---
<h4 style="margin-top:0; color:#1E40AF;">Inventory Table (<code>inventory_tbl</code>)</h4>
  <ul>
    <li><strong>item_id</strong> (string): Unique product identifier (e.g., SG001).</li>
    <li><strong>name</strong> (string): Style of sunglasses (e.g., Aviator, Round).</li>
    <li><strong>description</strong> (string): Text description of the product.</li>
    <li><strong>quantity_in_stock</strong> (int): Current stock available.</li>
    <li><strong>price</strong> (float): Price in USD.</li>
  </ul>
  <h4 style="margin-top:1em; color:#1E40AF;">Transactions Table (<code>transactions_tbl</code>)</h4>
  <ul>
    <li><strong>transaction_id</strong> (string): Unique identifier (e.g., TXN001).</li>
    <li><strong>customer_name</strong> (string): Name of the customer, or <code>OPENING_BALANCE</code> for initial entry.</li>
    <li><strong>transaction_summary</strong> (string): Short description of the transaction.</li>
    <li><strong>transaction_amount</strong> (float): Amount of money for this transaction.</li>
    <li><strong>balance_after_transaction</strong> (float): Running balance after applying the transaction.</li>
    <li><strong>timestamp</strong> (string): ISO-8601 formatted date/time of the transaction.</li>
  </ul>

### Plan

In [44]:
PROMPT = """You are a senior data assistant. PLAN BY WRITING PYTHON CODE USING TINYDB.

Database Schema & Samples (read-only):
{schema_block}

Execution Environment (already imported/provided):
- Variables: db, inventory_tbl, transactions_tbl  # TinyDB Table objects
- Helpers: get_current_balance(tbl) -> float, next_transaction_id(tbl, prefix="TXN") -> str
- Natural language: user_request: str  # the original user message

PLANNING RULES (critical):
- Derive ALL filters/parameters from user_request (shape/keywords, price ranges "under/over/between", stock mentions,
  quantities, buy/return intent). Do NOT hard-code values.
- Build TinyDB queries dynamically with Query(). If a constraint isn't in user_request, don't apply it.
- Be conservative: if intent is ambiguous, do read-only (DRY RUN).

TRANSACTION POLICY (hard):
- Do NOT create aggregated multi-item transactions.
- If the request contains multiple items, create a separate transaction row PER ITEM.
- For each item:
  - compute its own line total (unit_price * qty),
  - insert ONE transaction with that amount,
  - update balance sequentially (balance += line_total),
  - update the item’s stock.
- If any requested item lacks sufficient stock, do NOT mutate anything; reply with STATUS="insufficient_stock".

HUMAN RESPONSE REQUIREMENT (hard):
- You MUST set a variable named `answer_text` (type str) with a short, customer-friendly sentence (1–2 lines).
- This sentence is the only user-facing message. No dataframes/JSON, no boilerplate disclaimers.
- If nothing matches, politely say so and offer a nearby alternative (closest style/price) or a next step.

ACTION POLICY:
- If the request clearly asks to change state (buy/purchase/return/restock/adjust):
    ACTION="mutate"; SHOULD_MUTATE=True; perform the change and write a matching transaction row.
  Otherwise:
    ACTION="read"; SHOULD_MUTATE=False; simulate and explain briefly as a dry run (in logs only).

FAILURE & EDGE-CASE HANDLING (must implement):
- Do not capture outer variables in Query.test. Pass them as explicit args.
- Always set a short `answer_text`. Also set a string `STATUS` to one of:
  "success", "no_match", "insufficient_stock", "invalid_request", "unsupported_intent".
- no_match: No items satisfy the filters → suggest the closest in style/price, or invite a different range.
- insufficient_stock: Item found but stock < requested qty → state available qty and offer the max you can fulfill.
- invalid_request: Unable to parse essential info (e.g., quantity for a purchase/return) → ask for the missing piece succinctly.
- unsupported_intent: The action is outside the store’s capabilities → provide the nearest supported alternative.
- In all cases, keep the tone helpful and concise (1–2 sentences). Put technical details (e.g., ACTION/DRY RUN) only in stdout logs.

OUTPUT CONTRACT:
- Return ONLY executable Python between these tags (no extra text):
  <execute_python>
  # your python
  </execute_python>

CODE CHECKLIST (follow in code):
1) Parse intent & constraints from user_request (regex ok).
2) Build TinyDB condition incrementally; query inventory_tbl.
3) If mutate: validate stock, update inventory, insert a transaction (new id, amount, balance, timestamp).
4) ALWAYS set:
   - `answer_text` (human sentence, required),
   - `STATUS` (see list above).
   Also print a brief log to stdout, e.g., "LOG: ACTION=read DRY_RUN=True STATUS=no_match".
5) Optional: set `answer_rows` or `answer_json` if useful, but `answer_text` is mandatory.

TONE EXAMPLES (for `answer_text`):
- success: "Yes, we have our Classic sunglasses, a round frame, for $60."
- no_match: "We don’t have round frames under $100 in stock right now, but our Moon round frame is available at $120."
- insufficient_stock: "We only have 1 pair of Classic left; I can reserve that for you."
- invalid_request: "I can help with that—how many pairs would you like to purchase?"
- unsupported_intent: "We can’t refurbish frames, but I can suggest similar new models."

Constraints:
- Use TinyDB Query for filtering. Standard library imports only if needed.
- Keep code clear and commented with numbered steps.

User request:
{question}
"""

### Planning in Code 

In [45]:
def generate_llm_code(
    prompt: str,
    *,
    inventory_tbl,
    transactions_tbl,
    model: str = "gpt-5-mini",
    temperature: float = 0.2,
) -> str:
    """
    Ask the LLM to produce a plan-with-code response.
    Returns the FULL assistant content (including surrounding text and tags).
    The actual code extraction happens later in execute_generated_code.
    """
    schema_block = build_schema_block(inventory_tbl, transactions_tbl)
    prompt = PROMPT.format(schema_block=schema_block, question=prompt)

    resp = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {
                "role": "system",
                "content": "You write safe, well-commented TinyDB code to handle data questions and updates."
            },
            {"role": "user", "content": prompt},
        ],
    )
    content = resp.choices[0].message.content or ""
    
    return content  

> **Prompt:** “Do you have any round sunglasses in stock that are under $100?”

Before generating any code, let’s manually inspect the TinyDB tables to see if there are truly *round* frames (word-only match) and what their prices look like. Run the next cell to preview the inventory and highlight items that match the word-only “round” filter.

In [46]:
Item = Query()                    # Create a Query object to reference fields (e.g., Item.name, Item.description)

# Search the inventory table for documents where either the description OR the name
# contains the word "round" (case-insensitive). The check is done inline:
# - (v or "") ensures we handle None by converting it to an empty string
# - .lower() normalizes case
# - " round " enforces a crude word boundary (won't match "wraparound")
round_sunglasses = inventory_tbl.search(
    (Item.description.test(lambda v: " round " in ((v or "").lower()))) |
    (Item.name.test(        lambda v: " round " in ((v or "").lower())))
)

print(json.dumps(round_sunglasses, indent=2))

[
  {
    "item_id": "SG005",
    "name": "Classic",
    "description": "Classic round profile with minimalist metal frames, offering a timeless and versatile style that fits both casual and formal wear.",
    "quantity_in_stock": 10,
    "price": 60
  },
  {
    "item_id": "SG006",
    "name": "Moon",
    "description": "Oversized round style with bold plastic frames, evoking retro aesthetics with a modern twist.",
    "quantity_in_stock": 10,
    "price": 120
  }
]


Now let’s ask the model to **generate a plan in code**

In [47]:
prompt_round = "Do you have any round sunglasses in stock that are under $100?"

# Generate the plan-as-code (FULL content; may include <execute_python> tags)
full_content_round = generate_llm_code(
    prompt_round,
    inventory_tbl=inventory_tbl,
    transactions_tbl=transactions_tbl,
    model="gpt-5-mini",
    temperature=1.0,
)

print(full_content_round)

<execute_python>
# 1) Parse intent & constraints from user_request and build a TinyDB query to find matching inventory.
# 2) This request is a read-only query: "Do you have any round sunglasses in stock that are under $100?"
# 3) Follow the PLANNING RULES: derive filters from user_request (shape "round", price upper bound 100, in-stock).
# 4) Use TinyDB Query() to perform the search; no mutations will be made.

# Required environment variables provided:
# - db, inventory_tbl, transactions_tbl
# - get_current_balance, next_transaction_id
# - user_request (str)

# Step-by-step implementation with comments:
from tinydb import Query
import re

# 1) Initialize outputs and defaults
answer_text = ""
STATUS = ""
ACTION = "read"
SHOULD_MUTATE = False

# 2) Extract constraints from user_request
req = user_request.lower() if isinstance(user_request, str) else ""

# Detect shape keyword(s)
shape_terms = []
for term in ["round", "oval", "rectangle", "square", "aviator", "wayfarer", "retro", "oversi

Now we’ll define the function that **takes a plan produced by the model and runs it** safely:

- It **accepts either** the full LLM response (with `<execute_python>…</execute_python>`) **or** raw Python code.
- It **extracts** the executable block when needed.
- It runs the code in a **controlled namespace** (TinyDB tables + safe helpers only).
- It captures **stdout**, **errors**, and the model-set answer variables (`answer_text`, `answer_rows`, `answer_json`).
- It renders **before/after** table snapshots to make side effects explicit.

This is the “executor” that turns a **plan-as-code** into actions and a concise user-facing answer.


In [48]:
# --- Helper: extract code between <execute_python>...</execute_python> ---
def _extract_execute_block(text: str) -> str:
    """
    Returns the Python code inside <execute_python>...</execute_python>.
    If no tags are found, assumes 'text' is already raw Python code.
    """
    if not text:
        raise RuntimeError("Empty content passed to code executor.")
    m = re.search(r"<execute_python>(.*?)</execute_python>", text, re.DOTALL | re.IGNORECASE)
    return m.group(1).strip() if m else text.strip()


# ---------- 2) Code execution ----------
def execute_generated_code(
    code_or_content: str,
    *,
    db,
    inventory_tbl,
    transactions_tbl,
    user_request: Optional[str] = None,
) -> Dict[str, Any]:
    """
    Execute code in a controlled namespace.
    Accepts either raw Python code OR full content with <execute_python> tags.
    Returns minimal artifacts: stdout, error, and extracted answer.
    """
    # Extract code here (now centralized)
    code = _extract_execute_block(code_or_content)

    SAFE_GLOBALS = {
        "Query": Query,
        "get_current_balance": get_current_balance,
        "next_transaction_id": next_transaction_id,
        "user_request": user_request or "",
    }
    SAFE_LOCALS = {
        "db": db,
        "inventory_tbl": inventory_tbl,
        "transactions_tbl": transactions_tbl,
    }

    # Capture stdout from the executed code
    _stdout_buf, _old_stdout = io.StringIO(), sys.stdout
    sys.stdout = _stdout_buf
    err_text = None
    try:
        exec(code, SAFE_GLOBALS, SAFE_LOCALS)
    except Exception:
        err_text = traceback.format_exc()
    finally:
        sys.stdout = _old_stdout
    printed = _stdout_buf.getvalue().strip()

    # Extract possible answers set by the generated code
    answer = (
        SAFE_LOCALS.get("answer_text")
        or SAFE_LOCALS.get("answer_rows")
        or SAFE_LOCALS.get("answer_json")
    )


    return {
        "code": code,            # <- ya sin etiquetas
        "stdout": printed,
        "error": err_text,
        "answer": answer,
        "transactions_tbl": transactions_tbl.all(),  # For inspection
        "inventory_tbl": inventory_tbl.all(),  # For inspection
    }

In [49]:
# Execute the generated plan for the round-sunglasses question
result = execute_generated_code(
    full_content_round,          # the full LLM response you generated earlier
    db=db,
    inventory_tbl=inventory_tbl,
    transactions_tbl=transactions_tbl,
    user_request=prompt_round, # e.g., "Do you have any round sunglasses in stock that are under $100?"
)

In [50]:
print(result["code"])

# 1) Parse intent & constraints from user_request and build a TinyDB query to find matching inventory.
# 2) This request is a read-only query: "Do you have any round sunglasses in stock that are under $100?"
# 3) Follow the PLANNING RULES: derive filters from user_request (shape "round", price upper bound 100, in-stock).
# 4) Use TinyDB Query() to perform the search; no mutations will be made.

# Required environment variables provided:
# - db, inventory_tbl, transactions_tbl
# - get_current_balance, next_transaction_id
# - user_request (str)

# Step-by-step implementation with comments:
from tinydb import Query
import re

# 1) Initialize outputs and defaults
answer_text = ""
STATUS = ""
ACTION = "read"
SHOULD_MUTATE = False

# 2) Extract constraints from user_request
req = user_request.lower() if isinstance(user_request, str) else ""

# Detect shape keyword(s)
shape_terms = []
for term in ["round", "oval", "rectangle", "square", "aviator", "wayfarer", "retro", "oversized"]:
    if ter

In [51]:
print(result["answer"])

Yes, we have our Classic sunglasses in stock for $60.


This is the expected result based on our previous manual analysis.

### Query 2 

> **Request:** “Return 2 Aviator sunglasses I bought last week.”

Before generating the plan, let’s **inspect the current inventory** for the *Aviator* model.

In [52]:
Item = Query()                    # Create a Query object to reference fields (e.g., Item.name, Item.description)

# Query: fetch all inventory rows whose 'name' is exactly "Aviator".
# Notes:
# - This is a case-sensitive equality check. "aviator" won't match.
# - If you need case-insensitive matching, consider a .test(...) or .matches(...) with re.I.
aviators = inventory_tbl.search(
    (Item.name == "Aviator")
)

# Display the matched documents in a readable JSON panel
print(json.dumps(aviators))

[{"item_id": "SG001", "name": "Aviator", "description": "Originally designed for pilots, these teardrop-shaped lenses with thin metal frames offer timeless appeal. The large lenses provide excellent coverage while the lightweight construction ensures comfort during long wear.", "quantity_in_stock": 23, "price": 80}]


Inventory confirms one Aviator SKU in stock — **SG001 (Aviator)**: **23** units at **$80** each. Now let's generate a plan to answer the prompt.

In [53]:
prompt_aviator = "Return 2 Aviator sunglasses I bought last week."

# Generate the plan-as-code (FULL content; may include <execute_python> tags)
full_content_aviator = generate_llm_code(
    prompt_aviator,
    inventory_tbl=inventory_tbl,
    transactions_tbl=transactions_tbl,
    model="o4-mini",
    temperature=1,
)

print(full_content_aviator)

<execute_python>
# 1) Imports and initial setup
import re
from datetime import datetime
from tinydb import Query

# 2) Initialize variables
user_text = user_request  # the user message
ACTION = "mutate"
DRY_RUN = False
STATUS = None
answer_text = ""

# 3) Parse intent: look for 'return' action and extract quantity and item name
match = re.search(r'\breturn\s+(\d+)\s+(\w+)', user_text, re.IGNORECASE)
if not match:
    STATUS = "invalid_request"
    answer_text = "I can help with that—how many pairs would you like to return?"
    print(f"LOG: ACTION=read DRY_RUN=True STATUS={STATUS}")
else:
    qty = int(match.group(1))
    item_name = match.group(2)
    # 4) Query inventory for the item
    Item = Query()
    results = inventory_tbl.search(Item.name.test(lambda v, name=item_name: v.lower() == name.lower()))
    if not results:
        STATUS = "no_match"
        answer_text = f"We don’t have {item_name} in stock to return. Could you check the name and try again?"
        print(f"LOG: AC

In [54]:
# before return

transactions_tbl.all()

[{'transaction_id': 'TXN001',
  'customer_name': 'OPENING_BALANCE',
  'transaction_summary': 'Daily opening register balance',
  'transaction_amount': 500.0,
  'balance_after_transaction': 500.0,
  'timestamp': '2026-02-18T04:25:10.931112'}]

In [55]:
# Execute the generated plan for the round-sunglasses question
result = execute_generated_code(
    full_content_aviator,          # the full LLM response you generated earlier
    db=db,
    inventory_tbl=inventory_tbl,
    transactions_tbl=transactions_tbl,
    user_request=prompt_aviator, # e.g., "Return 2 aviator sunglasses I bought last week."
)

In [56]:
result["answer"]

'Your return of 2 Aviator sunglasses is complete and a refund of $160 has been processed.'

In [57]:
# after return

transactions_tbl.all()

[{'transaction_id': 'TXN001',
  'customer_name': 'OPENING_BALANCE',
  'transaction_summary': 'Daily opening register balance',
  'transaction_amount': 500.0,
  'balance_after_transaction': 500.0,
  'timestamp': '2026-02-18T04:25:10.931112'},
 {'transaction_id': 'TXN002',
  'customer_name': 'RETURN',
  'transaction_summary': 'Return 2 Aviator',
  'transaction_amount': -160,
  'balance_after_transaction': 340.0,
  'timestamp': '2026-02-18T04:26:06.294558'}]

A new transaction has been inserted for the Aviator sunglasses return.

In [58]:
# Stock-quantity check
Item = Query()                  

aviators = inventory_tbl.search(
    (Item.name == "Aviator")
)

print(json.dumps(aviators, indent=2))

[
  {
    "item_id": "SG001",
    "name": "Aviator",
    "description": "Originally designed for pilots, these teardrop-shaped lenses with thin metal frames offer timeless appeal. The large lenses provide excellent coverage while the lightweight construction ensures comfort during long wear.",
    "quantity_in_stock": 25,
    "price": 80
  }
]


The quantity in stock of the 'Aviator' sunglassed in now 25.

## Customer Service Agent

**Full workflow**

In [59]:
def customer_service_agent(
    question: str,
    *,
    db,
    inventory_tbl,
    transactions_tbl,
    model: str = "gpt-5-mini",
    temperature: float = 1.0,
    reseed: bool = False,
) -> dict:
    """
    End-to-end helper:
      1) (Optional) reseed inventory & transactions
      2) Generate plan-as-code from `question`
      3) Execute in a controlled namespace
      4) Render before/after snapshots and return artifacts

    Returns:
      {
        "full_content": <raw LLM response (may include <execute_python> tags)>,
        "exec": {
            "code": <extracted python>,
            "stdout": <plan logs>,
            "error": <traceback or None>,
            "answer": <answer_text/rows/json>,
            "inventory_after": [...],
            "transactions_after": [...]
        }
      }
    """
    # 0) Optional reseed
    if reseed:
        create_inventory()
        create_transactions()

    # 1) Show the question
    print(f"User Question:\n{question}\n{'-'*25}\n")

    # 2) Generate plan-as-code (FULL content)
    full_content = generate_llm_code(
        question,
        inventory_tbl=inventory_tbl,
        transactions_tbl=transactions_tbl,
        model=model,
        temperature=temperature,
    )
    print(f"Plan with Code (Full Response):\n{full_content}\n{'-'*25}\n")

    # 3) Before snapshots
    print(f"Inventory Table · Before:\n{json.dumps(inventory_tbl.all(), indent=2)}\n{'-'*25}\n")
    print(f"Transactions Table · Before:\n{json.dumps(transactions_tbl.all(), indent=2)}\n{'-'*25}\n")

    # 4) Execute
    exec_res = execute_generated_code(
        full_content,
        db=db,
        inventory_tbl=inventory_tbl,
        transactions_tbl=transactions_tbl,
        user_request=question,
    )

    # 5) After snapshots + final answer
    print(f"Plan Execution · Extracted Answer\n{exec_res["answer"]}\n{'-'*25}\n")
    print(f"Inventory Table · After\n{json.dumps(inventory_tbl.all(), indent=2)}\n{'-'*25}\n")
    print(f"Transactions Table · After\n{json.dumps(inventory_tbl.all(), indent=2)}\n{'-'*25}\n")


    # 6) Return artifacts
    return {
        "full_content": full_content,
        "exec": {
            "code": exec_res["code"],
            "stdout": exec_res["stdout"],
            "error": exec_res["error"],
            "answer": exec_res["answer"],
            "inventory_after": inventory_tbl.all(),
            "transactions_after": transactions_tbl.all(),
        },
    }


In [60]:
prompt = "I want to buy 3 pairs of classic sunglasses and 1 pair of aviator sunglasses."

out = customer_service_agent(
    prompt,
    db=db,
    inventory_tbl=inventory_tbl,
    transactions_tbl=transactions_tbl,
    model="o4-mini",
    temperature=1.0,
    reseed=True,   # set False to keep current state of the inventory and the transactions
)

User Question:
I want to buy 3 pairs of classic sunglasses and 1 pair of aviator sunglasses.
-------------------------

Plan with Code (Full Response):
<execute_python>
from tinydb import Query
from datetime import datetime

# 1) Parse user request for purchase quantities and item names.
import re
pattern = re.compile(r'(\d+)\s+pairs?\s+of\s+([A-Za-z]+)', re.IGNORECASE)
matches = pattern.findall(user_request or "")
purchase_requests = [(int(qty), name.strip()) for qty, name in matches]

# Prepare result tracking
insufficient = []
transactions_to_do = []

# 2) Validate each requested item and check stock
for qty, item_name in purchase_requests:
    # Find the item case-insensitive
    Item = Query()
    result = inventory_tbl.search(Item.name.test(lambda x, nm=item_name.lower(): x.lower() == nm))
    if not result:
        insufficient.append(("no_match", item_name))
        continue
    item = result[0]
    if item['quantity_in_stock'] < qty:
        insufficient.append(("insufficient_